In [8]:
import pandas as pd
from collections import Counter


In [9]:
def check_common_strings_across_columns(df):
    """
    检查DataFrame的不同列中是否有相同的字符串，并返回列名和具体重复值

    参数:
    df (pd.DataFrame): 数据框

    返回:
    list: 包含列名和具体重复值的列表
    """
    common_strings = []
    columns = df.columns.tolist()
    
    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col1 = columns[i]
            col2 = columns[j]
            col1_strings = set(df[col1].dropna().astype(str).tolist())
            col2_strings = set(df[col2].dropna().astype(str).tolist())
            common = col1_strings.intersection(col2_strings)
            if common:
                for value in common:
                    common_strings.append((col1, col2, value))
    
    return common_strings

def count_frequencies(df, column_name):
    """
    统计DataFrame中指定列的值的频率

    参数:
    df (pd.DataFrame): 数据框
    column_name (str): 要统计的列名

    返回:
    dict: 各个值的频率
    """
    frequency = Counter(df[column_name])
    return frequency

def change_key(d, old_key, new_key):
    if old_key in d:
        d[new_key] = d.pop(old_key)

In [10]:
# 指定CSV文件的路径
file_path = './data/failed_data_csv/外修基础数据_202212.csv'

# 读取CSV文件
data = pd.read_csv(file_path)

# 检查重复,重复值必须提前按列分开
# 要检查的列名列表
column_names = data.columns.tolist()

# 检查不同列中是否有相同的字符串
common_strings = check_common_strings_across_columns(data)

# 打印结果
for col1, col2, value in common_strings:
    print(f"列 '{col1}' 和列 '{col2}' 中有相同的字符串: {value}")

列 '维修人员' 和列 '站位' 中有相同的字符串: Others
列 'SN' 和列 'CRD' 中有相同的字符串: P1067482-05-A:SJBL22339P7H251
列 'SN' 和列 'CRD' 中有相同的字符串: P1671611-01-B:SJBL22337BGB3PW
列 '缺陷' 和列 '修理方法' 中有相同的字符串: others
列 'CRD' 和列 '站位' 中有相同的字符串: QC


In [11]:
# 替换重复值
data['缺陷'] = data['缺陷'].replace("others", "缺陷_Others")
data['修理方法'] = data['修理方法'].replace("others", "修理方法_Others")

In [12]:
# 统计每种缺陷对应的处理方法
temp_result = data.groupby('缺陷')['修理方法'].apply(list).to_dict()
failedtype_methods_frequency={key: dict(Counter(value)) for key, value in temp_result.items()}


In [13]:
# 统计频次并且按照频次值进行排序
failedtype_frequencies = dict(sorted(count_frequencies(data, '缺陷').items(), key=lambda item: item[1], reverse=True))
methods_frequencies = dict(sorted(count_frequencies(data, '修理方法').items(), key=lambda item: item[1], reverse=True))

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Graph

size_parameter=0.01
# 创建节点和边
nodes=[]
failedtype_nodes = [{"name": key, "symbolSize": [size_parameter*value, 20], "symbol": "rect","category": 1} for key, value in failedtype_frequencies.items()]
methods_nodes=[{"name": key, "symbolSize": [size_parameter*value, 20], "symbol": "rect","category": 2} for key, value in methods_frequencies.items()]

# 计算每列的最大节点大小
max_failedtype_size = max(size_parameter*value for key, value in failedtype_frequencies.items()) if failedtype_frequencies else 0
max_methods_size = max(size_parameter*value for key, value in methods_frequencies.items()) if methods_frequencies else 0

# 设置列间距和行间距
column_spacing = max(max_failedtype_size, max_methods_size)
row_spacing = 5

# 设置节点的坐标
nodes.append({"name": "缺陷", "symbolSize": 30, "x": 0, "y": 0, "category": 0})
for i, node in enumerate(failedtype_nodes):
    node["x"] = column_spacing
    node["y"] = i * row_spacing
    nodes.append(node)
for i, node in enumerate(methods_nodes):
    node["x"] = column_spacing*2
    node["y"] = i * row_spacing
    nodes.append(node)

num_rows_data = data.shape[0]
links=[]
failedtype_links = [{"source": "缺陷", "target": key, "value": value/num_rows_data, "emphasis": {"label": {"show": True, "formatter": "缺陷出现频率: {c}"}}} for key, value in failedtype_frequencies.items()]
methods_links = [
    {"source": key, "target": sub_key, "value": sub_value/sum(value.values()), "emphasis": {"label": {"show": True,  "formatter": "处理方法使用频率: {c}"}}}
    for key, value in failedtype_methods_frequency.items()
    for sub_key, sub_value in value.items()
]
links += failedtype_links
links += methods_links

# 定义分类
categories = [
    {"name": "分类","itemStyle": {"color": "red", "fontSize": 14, "fontWeight": "bold"}},
    {"name": "具体类型","itemStyle": {"color": "blue", "fontSize": 14, "fontWeight": "bold"}},
    {"name": "具体类型","itemStyle": {"color": "blue", "fontSize": 14, "fontWeight": "bold"}}
]

# 创建有向图
graph = Graph()

graph.add(
    "",
    nodes,
    links,
    categories=categories,
    layout="none",
    edge_label=opts.LabelOpts(is_show=False),
    is_draggable=True,
    edge_symbol=['none', 'arrow'],  # 设置边的形状为箭头
    edge_symbol_size=5,  # 设置箭头的大小
)
graph.set_global_opts(title_opts=opts.TitleOpts(title="【缺陷类型-修理方法】频率统计"))

# 渲染图表
graph.render("frequency_graph.html")
graph

'c:\\Users\\4097084\\code\\data_analysis\\frequency_graph.html'